In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install minisom

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from minisom import MiniSom

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

  Preparing metadata (setup.py) ... done
  Created wheel for minisom: filename=MiniSom-2.3.0-py3-none-any.whl size=9018 sha256=6992a985fe3a6b0272143fee80a2ce51b7ae07c423c3e047b5427708889a0f4f
  Stored in directory: /root/.cache/pip/wheels/d4/ca/4a/488772b0399fec45ff53132ed14c948dec4b30deee3a532f80
Successfully built minisom
/kaggle/input/runclustering/RunScores.csv
/kaggle/input/runclustering/consolidated.csv
/kaggle/input/runclustering/RouteUnitCoordinates.csv


The goal is to load up the run scores and dates, produce a list of lists, the sublist is the score for that time index.
Then the data is normalized, then it's run through a Self-Organizing Map for clustering.
Then we have to save the results of clustering for use later.

In [2]:
#scales 1, 6, 18?
def scaleCallingIndexes(lst) :
    #print("lst",lst)
    acc = 0
    acc += (1 * lst[0])
    acc += (6 * lst[1])
    acc += (18 * lst[2])
    return acc
    
# step 1 is to load and parse the run scores
runscores = pd.read_csv('../input/runclustering/RunScores.csv')
runscoreDict = {}
for index,row in runscores.iterrows() :
    newkey = row['RunID']
    newvalue = scaleCallingIndexes([row['1'], row['2'], row['3']])
    if newvalue is None :
        print("that didn't work")
        break
    runscoreDict[newkey] = newvalue

failedtransform = 0

def transformConsolidatedRow(row,route,year) :
    scorelist = []
    datelist = []
    finallist = []
    global failedtransform
    global runscoreDict
    if row['RunID1'] in runscoreDict :
        scorelist.append(runscoreDict[row['RunID1']])
        datelist.append(round(row['OrdinalDate1']))
    if row['RunID2'] in runscoreDict :
        scorelist.append(runscoreDict[row['RunID2']])
        datelist.append(round(row['OrdinarlDate2'])) # spelling mistake in input csv
    if row['RunID3'] in runscoreDict :
        scorelist.append(runscoreDict[row['RunID3']])
        datelist.append(round(row['OrdinalDate3']))
    if row['RunID4'] in runscoreDict :
        scorelist.append(runscoreDict[row['RunID4']])
        datelist.append(round(row['OrdinalDate4']))
    # numpy only does linear interpolation if the points are monotonically increasing, which I suppose we could do if we do it piecewise and reverse it if it's going the wrong way?
    numPieces = len(scorelist) - 1
    if numPieces == 0 :  # there is no data for run 283, the whole row should not be in consolidated.csv.  It's in Runs.csv but not Counts.csv.  So it's not in RunScores.csv
        #print("zero pieces",route,year)
        failedtransform += 1
        return None
    if numPieces < 0 :
        #print("negative pieces", route, year) # one row had no data for any of the three runs
        failedtransform += 1
        return None
    for x in range(numPieces) :
        #print("x is",x,"len scorelist",len(scorelist),"len runscoreDict",len(runscoreDict),scorelist)
        rise = scorelist[x+1] - scorelist[x]
        run = datelist[x+1] - datelist[x]
        finallist.append(scorelist[x])
        day = 1
        for y in range(datelist[x]+1, datelist[x+1]) :
            finallist.append(scorelist[x]+((day/run)*rise))
            day += 1
    finallist.append(scorelist[numPieces])
    if len(finallist) != 61 :
        # print(len(finallist),route,year,"uh oh, list not the right length",finallist) # this is due to missing data as well AFAICT
        failedtransform += 1
        return None
    return finallist
    
# step 2, load and parse the consolidated data
consolidated = pd.read_csv('../input/runclustering/consolidated.csv')
routeThenYear = {}
attempts = 0
for index, row in consolidated.iterrows() :
    route = str(row['RouteNumber'])
    year = str(row['SurveyYear'])
    # initialize if non-existent
    if route not in routeThenYear :
        routeThenYear[route] = {}
    if year not in routeThenYear[route] :
        attempts += 1
        traret = transformConsolidatedRow(row,route,year)
        if traret :
            routeThenYear[route][year] = traret
    #print(route,year,routeThenYear[route][year])
    #break
goodvalues = 0
for key, value in routeThenYear.items() :
    goodvalues = goodvalues + len(value)
print("failedtransform",failedtransform,'good values',goodvalues,'attempts',attempts)

failedtransform 379 good values 5739 attempts 6118


In [3]:
# now we have to normalize the data to use in clustering
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
#skipOuter = 3
#skipOuterCounter = 0
normalindex = 0
normalizedScoresOverTime = []
routeList = []
yearList = []
for key, value in routeThenYear.items() :
    for k2, v2 in value.items() : # dictionary of years mapped to 61-element lists
        llv2 = []
        for x in v2 : # each element has to be turned into a list
            llv2.append([x])
        t2 = MinMaxScaler().fit_transform(llv2)
        t2 = t2.reshape(len(t2))
        normalizedScoresOverTime.append(t2)
        routeList.append(key)
        yearList.append(k2)
        normalindex += 1
        #print(len(v2), len(llv2), len(t2))
        #print(v2)
        #print(llv2)
        #print(t2)
        #break
    #break
print("done normalizing",normalindex)

done normalizing 5739


In [5]:
# now we can run minisom
map_side_length = 40
asom = MiniSom(map_side_length, map_side_length, len(normalizedScoresOverTime[0]))
asom.random_weights_init(normalizedScoresOverTime)
asom.train(normalizedScoresOverTime, 50000)
print("training done")
# need to write all this to a file so we can run association analysis
xlist = []
ylist = []
for x in normalizedScoresOverTime :
    (xwin, ywin) = asom.winner(x)
    xlist.append(xwin)
    ylist.append(ywin)
outDataFrame = pd.DataFrame({
    'RouteNumber': routeList,
    'SurveyYear': yearList,
    'x': xlist,
    'y': ylist})
outDataFrame.to_csv('frogWinners.csv')
# print out scores to see if dense enough
for (row,column), value  in asom.win_map(normalizedScoresOverTime).items() :
    print(row,column,len(value))

training done
4 10 5
3 7 8
23 36 7
10 39 10
39 35 3
4 8 9
5 14 5
9 0 3
5 36 5
0 33 295
30 27 10
23 11 10
5 35 7
16 24 8
30 2 6
7 16 8
34 39 9
23 9 4
13 11 7
3 35 7
30 22 4
10 4 5
6 1 7
21 13 5
16 26 8
20 11 7
19 11 7
9 7 12
18 9 6
8 17 6
35 10 7
36 27 9
29 26 12
35 29 11
13 20 8
9 22 7
38 27 6
8 2 4
0 3 10
24 3 7
12 18 8
11 1 2
0 8 10
11 3 5
34 24 11
18 8 9
17 20 6
37 6 2
14 30 2
14 12 4
38 3 3
11 10 17
29 21 3
11 13 5
25 22 8
34 33 1
13 8 6
12 13 5
38 22 8
39 3 5
21 37 7
20 22 238
37 8 4
4 19 4
8 25 5
8 21 5
10 28 4
0 0 7
26 29 4
16 1 5
27 26 5
3 20 5
7 39 5
0 27 5
5 13 7
5 17 2
2 9 2
3 15 5
1 16 7
6 10 3
6 8 4
11 14 3
7 37 6
0 18 6
4 27 2
24 38 3
18 16 2
1 13 5
2 29 7
0 15 6
4 12 8
9 36 3
17 2 8
22 32 5
2 17 2
10 37 10
3 18 4
26 20 2
28 19 9
22 33 8
29 23 5
14 13 10
24 1 6
26 28 2
33 31 4
3 33 4
36 19 2
0 11 4
27 33 6
4 13 4
37 0 3
9 38 12
2 18 4
5 12 4
10 38 7
8 39 9
2 16 7
25 16 4
6 14 5
25 20 5
28 14 2
0 13 5
0 16 7
17 21 4
4 18 3
5 15 7
21 29 5
10 36 6
8 37 5
8 38 5
4 14 5
11 37 